In [ ]:
pip install datasets pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 11.2 MB/s eta 0:00:00


In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.9 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import pandas as pd
import torch
import pandas as pd
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import Trainer, TrainingArguments
import torch
import pandas as pd
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [ ]:
coqa_dataset = load_dataset("coqa")
coqa_dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/7199 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/500 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['source', 'story', 'questions', 'answers'],
        num_rows: 7199
    })
    validation: Dataset({
        features: ['source', 'story', 'questions', 'answers'],
        num_rows: 500
    })
})

In [ ]:
questions = []
for question in coqa_dataset['train']['questions']:
  questions.extend(question)

answers = []
for answer in coqa_dataset['train']['answers']:
  for ans in answer["input_text"]:
    if len(ans.split(" ")) > 5:
      answers.append(ans)


In [ ]:
dataset = pd.DataFrame(coqa_dataset['train'])

In [ ]:
len(answers)

11156

In [ ]:
data = {'Text': questions + answers, 'Label': [1] * len(questions) + [0] * len(answers)}

# Convert the dictionary to a DataFrame
qna_df = pd.DataFrame(data)

In [ ]:
qna_df

,Text,Label
0,When was the Vat formally opened?,1
1,what is the library for?,1
2,for what subjects?,1
3,and?,1
4,what was started in 2014?,1
...,...,...
119798,new middle-class homes and public buildings,0
119799,"George I, George II, George III, and George IV",0
119800,classical architecture of Greece and Rome,0
119801,to find out if the _Northland_ was at Upernivik,0


In [ ]:
qna_df.to_csv("qna_df.csv")

In [ ]:
# Load your dataset as a DataFrame
df = pd.read_csv("/content/qna_df.csv")

# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Load the pre-trained ELECTRA model and tokenizer
model_name = "google/electra-small-discriminator"
tokenizer = ElectraTokenizer.from_pretrained(model_name)
model = ElectraForSequenceClassification.from_pretrained(model_name)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def preprocess_data(df, tokenizer, max_length=128):
    tokenized_data = tokenizer(list(df["Text"]), truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    labels = torch.tensor(list(df["Label"]), dtype=torch.long)
    return tokenized_data, labels

train_data, train_labels = preprocess_data(train_df, tokenizer)
test_data, test_labels = preprocess_data(test_df, tokenizer)

In [ ]:
batch_size = 128
train_dataset = TensorDataset(train_data["input_ids"], train_data["attention_mask"], train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(test_data["input_ids"], test_data["attention_mask"], test_labels)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Define the training loop and fine-tune the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Li

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    print(epoch)
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

0
1
2


In [ ]:
# Evaluation
model.eval()
true_labels = []
predicted_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted_class.cpu().numpy())

In [ ]:
model_save_path = "electra_model.pt"

# Save the model's state dict
torch.save(model.state_dict(), model_save_path)

In [ ]:
# Generate a classification report
report = classification_report(true_labels, predicted_labels, target_names=["Answer", "Question"])
print(report)

              precision    recall  f1-score   support

      Answer       1.00      0.99      0.99      2150
    Question       1.00      1.00      1.00     21811

    accuracy                           1.00     23961
   macro avg       1.00      1.00      1.00     23961
weighted avg       1.00      1.00      1.00     23961



In [ ]:
t_questions = []
for question in coqa_dataset['validation']['questions']:
  t_questions.extend(question)

t_answers = []
for answer in coqa_dataset['validation']['answers']:
  for ans in answer["input_text"]:
    if len(ans.split(" ")) > 5:
      t_answers.append(ans)

t_data = {'Text': t_questions + t_answers, 'Label': [1] * len(t_questions) + [0] * len(t_answers)}

# Convert the dictionary to a DataFrame
qna_test_df = pd.DataFrame(t_data)

test_data_new, test_labels_new = preprocess_data(qna_test_df, tokenizer)

In [ ]:
test_dataset_new = TensorDataset(test_data_new["input_ids"], test_data_new["attention_mask"], test_labels_new)
test_loader_new = DataLoader(test_dataset_new, batch_size=batch_size)

In [ ]:
# Evaluation
model.eval()
true_labels = []
predicted_labels = []

with torch.no_grad():
    for batch in test_loader_new:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted_class.cpu().numpy())

In [ ]:
report = classification_report(true_labels, predicted_labels, target_names=["Answer", "Question"])
print(report)

              precision    recall  f1-score   support

      Answer       1.00      0.99      0.99       759
    Question       1.00      1.00      1.00      7983

    accuracy                           1.00      8742
   macro avg       1.00      1.00      1.00      8742
weighted avg       1.00      1.00      1.00      8742



In [ ]:
calls_df = pd.read_csv("/content/calls_df.csv",error_bad_lines=False)

<ipython-input-43-5916820faaab>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  calls_df = pd.read_csv("/content/calls_df.csv",error_bad_lines=False)
Skipping line 18850: expected 8 fields, saw 14



In [ ]:
calls_df.head()

,Unnamed: 0,id,call_id,Uuid,Agent,Customer,Call ID,Agent ID
0,0,73297518,180183496,71044ab3-0546-4882-a75d-3452db3b3f15,"Mhm K K K K K. Ok So yes, Mhm. What is...","hello,",71044ab3-0546-4882-a75d-3452db3b3f15,35.0
1,1,73297518,180183496,71044ab3-0546-4882-a75d-3452db3b3f15,Today this is shire with XX XX Home dog t...,"Uh, yes, it is. um, I'm currently busy aren...",71044ab3-0546-4882-a75d-3452db3b3f15,35.0
2,2,73297518,180183496,71044ab3-0546-4882-a75d-3452db3b3f15,"Yeah, once a good time to call you back,",Mhm. I'd say tomorrow morning,71044ab3-0546-4882-a75d-3452db3b3f15,35.0
3,3,73297518,180183496,71044ab3-0546-4882-a75d-3452db3b3f15,"tomorrow morning. Ok, definitely, we can do...",alright Thank you,71044ab3-0546-4882-a75d-3452db3b3f15,35.0
4,4,73297518,180183496,71044ab3-0546-4882-a75d-3452db3b3f15,"can also. Yeah, and I can also text you the...","alright, thank you so much.",71044ab3-0546-4882-a75d-3452db3b3f15,35.0


In [ ]:
t_data = {'Text': list(calls_df["Agent"]) + list(calls_df["Customer"]), 'Label': [1] * len(list(calls_df["Agent"])) + [0] * len(list(calls_df["Customer"]))}

In [ ]:
# Convert the dictionary to a DataFrame
qna_test_df = pd.DataFrame(t_data).sample(1000)

test_data_new, test_labels_new = preprocess_data(qna_test_df, tokenizer)

In [ ]:
test_dataset_new = TensorDataset(test_data_new["input_ids"], test_data_new["attention_mask"], test_labels_new)
test_loader_new = DataLoader(test_dataset_new, batch_size=batch_size)

In [ ]:
# Evaluation
model.eval()
true_labels = []
predicted_labels = []

with torch.no_grad():
    for batch in test_loader_new:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1)

        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted_class.cpu().numpy())

In [ ]:
report = classification_report(true_labels, predicted_labels, target_names=["Answer", "Question"])
print(report)

              precision    recall  f1-score   support

      Answer       0.51      0.57      0.54       481
    Question       0.56      0.49      0.52       519

    accuracy                           0.53      1000
   macro avg       0.53      0.53      0.53      1000
weighted avg       0.53      0.53      0.53      1000



In [ ]:
qna_test_df["Prediction"] = predicted_labels

In [ ]:
qna_test_df

,Text,Label,Prediction
1804027,if she'd like free at night shows like wid...,0,0
629715,"right? No, no, no, if you guys need to resch...",1,0
329286,ok,1,1
893079,polish up the little rough edges that he? u...,1,0
543581,well what we'll do is definitely make sure t...,1,0
...,...,...,...
1557036,"did he number? No. When I leave, I can't ...",0,1
359820,"Ok, that's great. I mean,",1,0
1295883,home unless it's in her home.,0,0
617423,Mhm.,1,1


In [ ]:
qna_test_df.to_csv("check.csv")